<a href="https://colab.research.google.com/github/junieberry/NLP-withPyTorch/blob/main/Yelp/03_Yelp_Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

공부한 코드 : https://github.com/rickiepark/nlp-with-pytorch/tree/main/chapter_3
함께한 재생목록 : https://www.youtube.com/watch?v=L3Chu2A5TyU

In [1]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

ERROR: torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


In [2]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm

## 3.6 레스토랑 리뷰 감성 분류하기

### 3.6.2  파이토치 데이터셋 이해하기

파이토치는 Dataset 클래스로 데이터셋을 추상화한다.

Dataset은 `__getitem()`과 `__len()` 메서드를 구현한다.

In [4]:
class ReviewDataset(Dataset):
  ## review_df (pandas.DataFrame) == 데이터셋
  ## vectorizer (ReviewVectorizer) == ReviewVectorizer 객체
  def __init__(self, review_df, vectorizer):
    self.review_df = review_df
    self._vectorizer = vectorizer

    self.train_df = self.review_df[self.review_df.split=='train']
    self.train_size = len(self.train_df)

    self.val_df = self.review_df[self.review_df.split=='val']
    self.validation_size = len(self.val_df)

    self.test_df = self.review_df[self.review_df.split=='test']
    self.test_size = len(self.test_df)

    self._lookup_dict = {'train': (self.train_df, self.train_size),
                          'val': (self.val_df, self.validation_size),
                         'test': (self.test_df, self.test_size)}

    self.set_split('train')
  

  ## classmethod는 static과 같다고 볼 수 있따..
  @classmethod

  ## 데이터셋 로드하고 새 RevewVectorizer을 만든당
  ## review_csv (str) == 데이터셋 위치
  ## return == ReviewDataset의 인스턴스
  def load_dataset_and_make_vectorizer(cls, review_csv):
    review_df = pd.read_csv(review_csv)
    train_review_df = review_df[review_df.split=='train']
    return cls(review_df,ReviewVectorizer.from_dataframe(train_review_df))

  @classmethod
  ## 데이터셋 로드하고 ReviewVectorizer 재사용
  ## review_csv (str) == 데이터셋 위치
  ## vectorizer_filepath (str) == ReviewVectorizer 객체의 저장 위치
  ## return == ReviewDataset의 인스턴스

  def load_Dataset_and_load_vectorizer(cls, review_csv, vectorizer_filepath):
    review_df = pd.read_csv(review_csv)
    vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
    return cls(review_df, vectorizer)
  
  @staticmethod
  ## 파일에서 ReviewVectorizer 로드
  ## vectorizer_filepath (str) == 직렬화된 ReviewVectorizer 객체의 위치
  ## return == ReviewVectorizer의 인스턴스
  def load_vectorizer_only(vectorizer_filepath):
    with open(vectorizer_filepath) as fp:
        return ReviewVectorizer.from_serializable(json.load(fp))
  
  ## ReviewVectorizer 객체를 json 형태로 저장
  ## vectorizer_filepath (str) == ReviewVectorizer의 저장 위치
  def save_vectorizer(self, vectorizer_filepath):
    with open(vectorizer_filepath, "w") as fp:
      json.dump(self._vectorizer.to_serializable(), fp)
  
  ## 백터 변환 객체 반환
  def get_vectorizer(self):
    return self._vectorizer
  

  ## 데이터 프레임에 있는 열을 사용해 분할 세트 선택
  ## split (str) == "train", "val", "test" 중 하나
  def set_split(self, split="train"):
    self._target_split = split
    self._target_df, self._target_size = self._lookup_dict[split]

  def __len__(self):
    return self._target_size

  
  ## 데이터셋의 진입 메서드
  ## index (int) == 데이터 포인트 인덱스
  ## return == x_data와 y_target으로 이루어진 딕셔너리
  def __getitem__(self, index):
    row = self._target_df.iloc[index]

    review_vector = self._vectorizer.vectorize(row.review)
    rating_index = self._vectorizer.rating_vocab.lookup_token(row.rating)

    return {'x_data' : review_vector, 'y_target':rating_index}
  
  ## 배치 크기를 받아 배치 개수를 반환
  def get_num_batches(self, batch_size):
    return len(self)//batch_size


### 3.6.3 Vocabulary, Vectorizer, Dataloader

토큰을 정수에 매핑하고, 이 매핑을 각 데이터 포인트에 적용해 백터 형태로 변환

그 후 벡터로 변환한 데이터 포인트를 모델을 위해 미니배치로 모음

Vocabulary = 정수-토큰 매핑

Vocabulary = 텍스트 토큰과 클래스 레이블 정수로 매핑

Vectorizer = 어휘 사전 캡츌화, 문자열 데이터를 수치 벡터로 변환

DataLoader = 개별 벡터 데이터 포인트를 미니배치로

**Vocabulary**

토큰을 정수로 매핑!

- 토큰이 추가되면 자동으로 인덱스 증가
- UNK 토큰 사용


- `add_token` = vocabulary에 새로운 토큰 추가
- `lookup_token` = 토큰에 해당하는 인덱스 추가
- `lookup_index` = 특정 인덱스에 해당하는 토큰 추가



In [8]:
class Vocabulary(object):

  ##token_to_idx (dict): 기존 토큰-인덱스 매핑 딕셔너리
  ## add_unk (bool): UNK 토큰을 추가할지 지정하는 플래그
  ## unk_token (str): Vocabulary에 추가할 UNK 토큰
  def __intit__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):

    if token_to_idx is None:
      self._token_to_idx = {}
    
    self._token_to_idx = token_to_idx

    ## ???
    self._token_to_idx = {idx:token
                          for token, idx in self._token_to_idx.items()}
    self._add_unk = add_unk
    self._unk_token = unk_token

    self.unk_index = -1
    if add_unk:
      self.unk_index = self.add_token(unk_token)
  
  ## 직렬화할 수 있는 딕셔너리를 반환
  def to_serializable(self):
    return { 'token_to_idx': self._token_to_idx,
            'add_unk': self._add_unk,
            'unk_token': self._unk_token
    }
  
  @classmethod
  ## 직렬화된 딕셔너리에서 Vocabulary 객체를 만든다.
  def from_serializable(cls, contents):
    return cls(**contents)
  
  ## 토큰 추가하고 매핑 딕셔너리를 업데이트
  ## token (str) == Vocabulary에 추가할 토큰
  ## return == 토큰의 index
  def add_token(self, token):
    if token in self._token_to_idx:
      index = self._token_to_idx[token]
    else:
      index = len(self._token_to_idx)
      self._token_to_idx[token] = index
      self._idx_to_token[index] = token
    return index


  ## 토큰들을 추가하고 매핑 딕셔너리를 업데이트
  ## tokens (list) == 문자열 토큰 리스트
  ## return == 토큰들의 indices
  def add_many(self, tokens):
    return [self.add_token(token) for token in tokens]


  ## 토큰에 대응하는 인덱스 추출, 토큰이 없으면 UNK 인덱스 반환
  ## token (str) == 찾을 토큰
  ## index (int) == 토큰에 해당하는 인덱스
  def lookup_token(self, token):
        
        ## ??????
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

  ## 인덱스에 해당하는 토큰 반환
  ## index (int) == 찾을 인덱스
  ## return = 인덱스의 토큰
  def lookup_index(self, index):
    if index not in self._idx_to_token:
      raise KeyError("Vocabulary에 인덱스(%d)가 없습니다." % index)
    return self._idx_to_token[index]

  def __str__(self):
    return "<Vocabulary(size=%d)>" % len(self)

  def __len__(self):
    return len(self._token_to_idx)



**Vectorizer**

입력 데이터의 토큰을 순회하며 각 토큰을 정수로 바꿔 벡터로 만든다.

이때 벡터들의 크기는 모두 같아야한다!


 `from_dataframe()` = Dataframe을 순회하며 Vectorizer 클래스 초기화
  1. 데이터셋에 있는 모든 토큰의 빈도수 카운트
  2. cutoff보다 빈도수가 높은 Vocabulary 객체만 남김

`vectorize()` = Vectorizer 클래스의 핵심 기능 캡슐화

  - 매개변수로 리뷰 문자열을 받고 리뷰의 벡터 표현을 반환 (원핫 벡터)

  - 이때 벡터 표현은 희소 표현이며 BoW 방식이다.

In [14]:
class ReviewVectorizer(object):

  ## review_vocab (Vocabulary) == 단어를 정수에 매핑하는 Vocabulary
  ## rating_vocab (Vocabulary) == 클래스 레이블을 정수에 매핑하는 Vocabulary
  def __init__(self, review_vocab, rating_vocab):
    self.review_vocab = review_vocab
    self.rating_vocab = rating_vocab
  

  ## 매개변수로 받은 리뷰를 원핫 벡터로 변환시킴
  ## review (str) == 리뷰
  ## return = 변환된 원핫 벡터
  def vectorize(self, review):
    one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)

    for token in review.split(" "):

      ## string.punctuation == !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
      if token not in string.punctuation:
        one_hot[self.review_vocab.lookup_token(token)] = 1
    return one_hot
  
  @classmethod
  ## 데이터셋 데이터프레임에서 Vectorizer 객체를 만든다
  ## review_df (pandas.DataFrame) == 리뷰 데이터셋
  ## cutoff (int) == 빈도 기반 필터링 설정값
  ## return == ReviewVectorizer 객체
  def from_dataframe(cls, review_df, cutoff=25):
    review_vocab = Vocabulary(add_unk=True)
    rating_vocab = Vocabulary(add_unk=False)

    ## 점수 추가
    for rating in sorted(set(review_df.rating)):
      rating_vocab.add_token(rating)
    
    ## cutoff보다 많이 등장하는 단어 추가
    word_couts = Counter()
    for review in review_df.review:
      for word in review.split(" "):
        if word not in string.punctuation:
          word_counts[word] += 1
    
    for word, count in word_counts.items():
      if count > cutoff:
        review_vocab.add_token(word)
    
    return cls(review_vocab, rating_vocab)

    """ 직렬화된 딕셔너리에서 ReviewVectorizer 객체를 만듭니다
      
      매개변수:
          contents (dict): 직렬화된 딕셔너리
      반환값:
           ReviewVectorizer 클래스 객체
      """
  @classmethod
  def from_serializable(cls, contents):
      
    review_vocab = Vocabulary.from_serializable(contents['review_vocab'])
    rating_vocab =  Vocabulary.from_serializable(contents['rating_vocab'])

    return cls(review_vocab=review_vocab, rating_vocab=rating_vocab)

  def to_serializable(self):
    """ 캐싱을 위해 직렬화된 딕셔너리를 만듭니다
        
    반환값:
        contents (dict): 직렬화된 딕셔너리
    """
    return {'review_vocab': self.review_vocab.to_serializable(), 'rating_vocab': self.rating_vocab.to_serializable()}

In [9]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


**DataLoader**

미니 배치로 모아줌

Dataset, batch_size 등을 매개변수로 받아 순회

https://dojang.io/mod/page/view.php?id=2412

In [15]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):

    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)


    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

### 3.6.4 퍼센트론 분류기

